# Scopus Authors
### This is an academic project for authors data collection

In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import mysql.connector
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import userip as ui # this is the module I created that includes classes for getting the list of different user agents and ips
from itertools import cycle
import traceback

In [ ]:
#MySql connector
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="shahab",
    database="datascopus")

In [ ]:
# we check the database connection
if mydb:
    print("connected")
else:
    print("NOT connected")

In [2]:
a = ui.UserAgent()
b = ui.Ip()
users = a.get_list()
ips = b.get_ip()

In [3]:
print("number of ips: {}".format(len(ips))) #sometimes there is no ips that matches our conditions

number of ips: 8


In [4]:
#ips[0]

In [5]:
print("number of user agents: {}".format(len(users)))

number of user agents: 10839


In [6]:
users[100] #test

'Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.1; AOLBuild 4334.5000; Windows NT 5.1; Trident/4.0)'

In [ ]:
mycursor = mydb.cursor()
df = pd.read_csv("authors.csv") #authors CSV file
df = df.dropna(axis=0)
df.info()

In [ ]:
authors_array = df.loc[:, "author_id"].values
authors_list = authors_array.tolist()

In [ ]:
proxy_pool = cycle(ips)
user_pool = cycle(users)

### we can run loop over some ips and user agents. (some of the user agents do not work)
### It is better to test the ones that work in advance and then create a list and run a loop over that.
### I created a module that extracts 10839 user agents and some ips (if available) from these two websites:
### 1)  https://free-proxy-list.net
### 2)  www.useragentstring.com

In [ ]:
for auth in authors_list:
    url = "https://www.scopus.com/authid/detail.uri?authorId={}".format(auth)
    print(auth)
    a = "wrong"
    while a != "right":
        try:
            response = requests.get(url,headers={'User-Agent': "Mozilla/5.0 (compatible; U; ABrowse 0.6;  Syllable) AppleWebKit/420+ (KHTML, like Gecko)"})
            #response = requests.get(url,headers={'User-Agent': next(user_pool)})
            # response = requests.get(url,headers={'User-Agent': next(user_pool)}, proxies = {"https" : next(proxy_pool)})
            a = "right" 
        except:
            continue
    
    page_soup = BeautifulSoup(response.content, "html.parser")
    first = page_soup.findAll("h2", {"class" : "wordBreakWord"})
    full_name = first[0].text.replace(",\xa0", " ").replace("\r", " ").replace("Is this you? Claim profile  Opens in new window", " ").strip()
    print(full_name)
    
    other_names = page_soup.findAll("div", {"id" : "otherNameFormatBadges"})
    span_list = other_names[0].findAll("span")
    other_names = []
    for i in span_list:
        other_names.append(i.text.strip())
    others = "|".join(other_names).replace("|", "| ") 
    t = len(other_names)
    print(others)
        

    university = page_soup.findAll("div", {"class" : "authAffilcityCounty"})[0].text.strip().replace("\r", " ")
    print(university)

    subject_areas = page_soup.findAll("span", {"class" : "badges"})
    subject_areas = subject_areas[t:]
    

    areas = []
    for i in range(len(subject_areas)):
        areas.append(subject_areas[i].text.strip().replace(",", ""))

    subjects = ",".join(areas).replace(",", ", ")
    print(subjects)

    documents_by_author = page_soup.findAll("span", {"class" : "fontLarge pull-left"})[0].text
    print(documents_by_author)

    total_citation_number = page_soup.findAll("span", {"class" : "fontLarge darkGrayText"})[0].text
    print(total_citation_number)

    total_document = page_soup.findAll("div", {"class" : "lightGreyText"})[0].findAll("span")[1].text
    print(total_document)

    h_index = page_soup.findAll("span", {"class" : "fontLarge"})[0].text
    print(h_index)
    
    sql = """INSERT INTO authors(full_name, others, university, subjects, document_by_author, total_citation_numebr, total_document, h_index) VALUES (
    "{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}")""".format(full_name, others, university, subjects, int(documents_by_author),
                                                       int(total_citation_number), int(total_document), int(h_index))
    
    try:
        mycursor.execute(sql)
        mydb.commit()
    except:
        mydb.rollback()
        print("NOT connected")
    time.sleep(20) # 20 seconds sleep
    
    
    